
### Цель работы

В данном ноутбуке используется LangChain и OpenAI для анализа запросов и оценки их времени выполнения.
Мы создадим функцию для трассировки и валидации ответов, получаемых от LLM (Large Language Models),
и протестируем производительность модели на простом примере.


In [ ]:
!pip install langchain langchain_community openai faiss-cpu tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/14


### Установка необходимых библиотек

Для корректной работы ноутбука необходимо установить несколько библиотек: `langchain`, `openai`, `faiss-cpu`, и другие.
Они будут использованы для генерации ответов и обработки запросов к языковым моделям.


In [ ]:
# Импорт необходимых библиотек для работы с LLM и FAISS
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import logging
import os
import time

# Настраиваем логирование для вывода в консоль (Colab)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', handlers=[logging.StreamHandler()])
logger = logging.getLogger(__name__)

# Устанавливаем ключ API для OpenAI
os.environ["OPENAI_API_KEY"] = ""


In [ ]:
# Инициализация модели GPT-3.5-turbo для работы с чатами
def initialize_llm():
    logger.info("Настраиваем модель ChatOpenAI с GPT-3.5-turbo...")
    try:
        llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"))
        logger.info("Chat LLM успешно настроена.")
        return llm
    except Exception as e:
        logger.error(f"Ошибка при инициализации модели: {e}")
        raise e

llm = initialize_llm()


<ipython-input-3-0626133f1975>:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"))


In [ ]:
# Создание шаблона для диалога с нейросотрудником
def create_chat_prompt():
    logger.info("Создаем шаблон для вопросов...")
    try:
        chat_prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="Ты эксперт по машинному обучению."),
            HumanMessage(content="{question}")
        ])
        logger.info("Шаблон для вопросов успешно создан.")
        return chat_prompt
    except Exception as e:
        logger.error(f"Ошибка при создании шаблона: {e}")
        raise e

chat_prompt = create_chat_prompt()


In [ ]:
# Инициализация FAISS и подготовка базы знаний
def initialize_faiss(embeddings, documents):
    logger.info("Инициализируем FAISS и загружаем базу знаний...")
    try:
        # Преобразуем текст документов в эмбеддинги
        texts = [doc['text'] for doc in documents]
        ids = [doc['id'] for doc in documents]

        # Создаем векторное хранилище с использованием FAISS
        knowledge_base = FAISS.from_texts(texts, embeddings, metadatas=[{"id": doc["id"]} for doc in documents])
        logger.info("Документы успешно загружены в FAISS.")
        return knowledge_base
    except Exception as e:
        logger.error(f"Ошибка при инициализации FAISS: {e}")
        raise e

documents = [
    {"text": "Документация по Scikit-Learn", "id": "1"},
    {"text": "Туториал по нейронным сетям", "id": "2"}
]

embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
knowledge_base = initialize_faiss(embeddings, documents)


<ipython-input-5-9fa5ee9da349>:22: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))


In [ ]:
# Функция для трассировки запроса и ответа модели
def trace_query(query, llm, chat_prompt):
    logger.info(f"Трассировка запроса: '{query}'")
    start_time = time.time()

    try:
        prompt = chat_prompt.format_prompt(question=query).to_messages()
        response = llm(messages=prompt)
        elapsed_time = time.time() - start_time
        logger.info(f"Запрос успешно обработан за {elapsed_time:.2f} секунд. Ответ модели: {response}")
        return response, elapsed_time
    except Exception as e:
        logger.error(f"Ошибка при обработке запроса: {e}")
        raise e

# Пример трассировки запроса
query = "Что такое машинное обучение?"
response, time_taken = trace_query(query, llm, chat_prompt)


<ipython-input-6-6df8b574c67d>:8: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm(messages=prompt)


In [ ]:
# Функция для валидации ответа на основе знаний
def validate_response(query, response, knowledge_base):
    logger.info(f"Валидация ответа для запроса: '{query}'")
    try:
        relevant_docs = knowledge_base.search(query, search_type="similarity")
        if not relevant_docs:
            logger.warning("Не найдено релевантных документов в базе знаний. Возможна галлюцинация.")
            return False

        logger.info(f"Найдено {len(relevant_docs)} релевантных документов.")
        for doc in relevant_docs:
            logger.info(f"Документ ID: {doc.metadata['id']}, Текст: {doc.page_content}")
        return True
    except Exception as e:
        logger.error(f"Ошибка при валидации ответа: {e}")
        return False

# Пример валидации ответа
is_valid = validate_response(query, response, knowledge_base)
if not is_valid:
    logger.error("Ответ не прошел валидацию.")
else:
    logger.info("Ответ успешно прошел валидацию.")


In [ ]:
# Основная функция для обработки запросов и валидации
def process_query(query, llm, chat_prompt, knowledge_base):
    logger.info(f"Начинаем обработку запроса: '{query}'")
    response, time_taken = trace_query(query, llm, chat_prompt)

    if not validate_response(query, response, knowledge_base):
        logger.error(f"Галлюцинация обнаружена при обработке запроса '{query}'")
    else:
        logger.info(f"Запрос '{query}' успешно обработан и валидирован.")

    return response

# Пример обработки запроса
query = "Что такое нейронные сети?"
process_query(query, llm, chat_prompt, knowledge_base)


AIMessage(content='Привет! Чем могу помочь?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 31, 'total_tokens': 45, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-74a59c96-eb34-4474-8fff-688e22f6de86-0')

In [ ]:

# Функция для анализа производительности запроса
# Аргументы:
# query: строка, содержащая текст запроса
# llm: модель языкового моделирования, используемая для генерации ответа
# chat_prompt: подсказка для чат-бота, если она используется
# knowledge_base: база знаний для валидации ответов
def analyze_performance(query, llm, chat_prompt, knowledge_base):
    # Логирование начала анализа времени
    logger.info("Анализируем время выполнения запросов...")

    # Запускаем трассировку выполнения запроса
    response, time_taken = trace_query(query, llm, chat_prompt)

    # Выводим информацию о времени выполнения
    logger.info(f"Время выполнения запроса: {time_taken:.2f} секунд.")

    # Проверяем корректность полученного ответа с базой знаний
    if validate_response(query, response, knowledge_base):
        logger.info(f"Запрос '{query}' прошел валидацию и успешно обработан.")
    else:
        logger.error(f"Ошибка: Запрос '{query}' не прошел валидацию.")

    # Возвращаем время выполнения
    return time_taken



### Пример использования функции

После настройки окружения, вы можете протестировать производительность модели с помощью запроса.
Ниже приведён пример запроса о градиентном спуске и оценка времени выполнения запроса.
```python
query = "Расскажи про градиентный спуск"
time_taken = analyze_performance(query, llm, chat_prompt, knowledge_base)
print(f"Время выполнения запроса: {time_taken:.2f} секунд.")
```



### Вывод

Данный ноутбук позволяет трассировать запросы и проверять время их выполнения с использованием языковых моделей.
На основе результатов можно оптимизировать производительность модели, а также валидировать корректность ответов.
